<a href="https://colab.research.google.com/github/Idan-Alter/OU-22961-Deep-Learning/blob/main/22961_6_1_working_with_text_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

     |████████████████████████████████| 325 kB 7.0 MB/s 
     |████████████████████████████████| 77 kB 3.4 MB/s 
     |████████████████████████████████| 212 kB 19.5 MB/s 
     |████████████████████████████████| 136 kB 39.0 MB/s 
     |████████████████████████████████| 1.1 MB 13.6 MB/s 
     |████████████████████████████████| 127 kB 43.8 MB/s 
     |████████████████████████████████| 271 kB 6.4 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 55.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import torch
from torch import nn
import datasets as ds
from pprint import pprint

In [ ]:
dataset = ds.load_dataset("glue", "sst2")
dataset

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [ ]:
pprint(dataset["train"])
pprint(dataset["train"][1:3])

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})
{'idx': [1, 2],
 'label': [0, 1],
 'sentence': ['contains no wit , only labored gags ',
              'that loves its characters and communicates something rather '
              'beautiful about human nature ']}


In [ ]:
sentence_list = dataset["train"]["sentence"]
labels_list   = dataset["train"]["label"]
print(sentence_list[1], labels_list[1], sep="\n")

contains no wit , only labored gags 
0


Tokenization

In [ ]:
sentence = dataset["train"]["sentence"][1]
print(sentence,type(sentence))
print(sentence.split())

contains no wit , only labored gags  <class 'str'>
['contains', 'no', 'wit', ',', 'only', 'labored', 'gags']


Createa Vocabulary

In [ ]:
from torchtext.vocab import build_vocab_from_iterator
tokenizer = lambda x: x.split()
tokenized = list(map(tokenizer, sentence_list))

vocab = build_vocab_from_iterator(tokenized)
print(vocab(sentence.split()))
print(vocab("one two three".split()))

[2923, 60, 329, 1, 88, 1992, 548]
[28, 128, 356]


In [ ]:
print(vocab.get_itos()[0:10]) # integer to string

['the', ',', 'a', 'and', 'of', '.', 'to', "'s", 'is', 'that']


In [ ]:
pprint(labels_list[1:3])
pprint(sentence_list[1:3])
print(*tokenized[1:3], sep="\n")

[0, 1]
['contains no wit , only labored gags ',
 'that loves its characters and communicates something rather beautiful about '
 'human nature ']
['contains', 'no', 'wit', ',', 'only', 'labored', 'gags']
['that', 'loves', 'its', 'characters', 'and', 'communicates', 'something', 'rather', 'beautiful', 'about', 'human', 'nature']


Out of Vocabulary

In [ ]:
vocab("hello world".split())

RuntimeError: ignored

In [ ]:
vocab = build_vocab_from_iterator(tokenized,
                                specials=["<UNK>"], min_freq=5)
vocab.set_default_index(0)
vocab("hello world".split())

[0, 152]

In [ ]:
print(len(vocab))
print(vocab.get_itos()[0:15]) # integer to string
print(vocab(["the","me","you","pizza","teleport","q!@אבג"]))

11222
['<UNK>', 'the', ',', 'a', 'and', 'of', '.', 'to', "'s", 'is', 'that', 'in', 'it', 'as', 'with']
[1, 238, 21, 5003, 0, 0]


list of strings to integer tensor 

In [ ]:
stoi = lambda x: torch.tensor(vocab(x)) # string to integer
integer_tokens = list(map(stoi,tokenized))
print(integer_tokens[1])

tensor([2924,   61,  330,    2,   89, 1993,  549])


#Learnable Embeddings

In [ ]:
embedding_layer = nn.Embedding(len(vocab),3)
integer = stoi(["word"])
print(integer)
print(embedding_layer(integer))

tensor([1234])
tensor([[ 0.6333,  0.1598, -0.8170]], grad_fn=<EmbeddingBackward0>)


In [ ]:
W = embedding_layer.weight
print(W.size())
print(W[integer,:])

torch.Size([11222, 3])
tensor([[ 0.6333,  0.1598, -0.8170]], grad_fn=<IndexBackward0>)


In [ ]:
embedded_tokens = embedding_layer(integer_tokens[0])
print(integer_tokens[0])
print(embedded_tokens)


tensor([ 4575,    93,     0,    36,     1,  7746, 10013])
tensor([[-0.7823, -0.5872,  1.0293],
        [-0.8385,  0.2898,  1.6136],
        [ 0.9615, -0.2704, -0.4826],
        [-0.2764, -1.5677, -0.2256],
        [ 2.1826,  0.4882, -0.9949],
        [-0.1431,  0.3114, -0.7551],
        [ 0.5556, -0.2874, -0.1649]], grad_fn=<EmbeddingBackward0>)


#Pretrained embeddings

In [ ]:
from torchtext.vocab import GloVe
glove_embedder = GloVe(name='6B', dim=50)
embedded = glove_embedder.get_vecs_by_tokens(["It's", 'cool'])
print(embedded.size(), embedded.dtype,
      embedded.requires_grad, sep="\n")

torch.Size([2, 50])
torch.float32
False
